In [1]:
from pygraylog.graylogapi import GraylogAPI

In [2]:
graylog_con={
        "user":"merger",
        "password":"",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [3]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [4]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

In [5]:
res=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [12]:
res

u'{"query":"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\"a2:2a:0c:40\\"","built_query":"{\\"from\\":0,\\"size\\":150,\\"query\\":{\\"bool\\":{\\"must\\":{\\"query_string\\":{\\"query\\":\\"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\\\\\"a2:2a:0c:40\\\\\\"\\",\\"allow_leading_wildcard\\":false}},\\"filter\\":{\\"bool\\":{\\"must\\":{\\"range\\":{\\"timestamp\\":{\\"from\\":\\"2018-01-12 04:00:00.000\\",\\"to\\":\\"2018-01-12 11:00:00.000\\",\\"include_lower\\":true,\\"include_upper\\":true}}}}}}},\\"sort\\":[{\\"timestamp\\":{\\"order\\":\\"desc\\"}}]}","used_indices":[{"index_name":"graylog_36","begin":"1970-01-01T00:00:00.000Z","end":"1970-01-01T00:00:00.000Z","calculated_at":"2018-01-11T21:10:13.698Z","took_ms":0}],"messages":[{"highlight_ranges":null,"message":{"e212_imsi":"425191201068102","m3ua_DPC":"7138","sccp_called_nai":"4","sccp_called_digits":"97255991201068